In [10]:
from Utils import split_dataset_casia_wf, create_data_loader, slplit_dataset_digi_face
from MoEViT import MoEViT, MoEViTConfig
import torch
from torchsummary import summary
#from SigleDeviceTraining import train
torch.cuda.empty_cache()
%env CUDA_LAUNCH_BLOCKING=0

env: CUDA_LAUNCH_BLOCKING=0


In [ ]:
#split_dataset_casi_wf(base_folder='casia-web-face-dataset', validation_percentage=0.15, num_identification_imgs=500)
#slplit_dataset_digi_face(base_folder='Digi-Face', validation_percentage=0.1, num_identification_imgs=500)

In [2]:
train_loader = create_data_loader('data/data_t/train', 112, 10)

val_loader = create_data_loader('data/data_t/validation', 112, 10)

id_imgs_loader = create_data_loader('data/casia-web-face-dataset/identification_imgs', 112, 10)

In [11]:
current_device = 'cuda'

config = MoEViTConfig(device=current_device)

model = MoEViT(config)
model.to(current_device)

summary(model, input_size = (3, 112, 112), batch_size = 8, device='cuda', verbose=0)

Layer (type:depth-idx)                   Param #
├─ModuleDict: 1-1                        --
|    └─Conv2d: 2-1                       113,664
|    └─Dropout: 2-2                      --
|    └─ModuleList: 2-3                   --
|    |    └─EncoderBlock: 3-1            30,091,788
|    |    └─EncoderBlock: 3-2            30,091,788
|    |    └─EncoderBlock: 3-3            30,091,788
|    |    └─EncoderBlock: 3-4            30,091,788
|    |    └─EncoderBlock: 3-5            30,091,788
|    |    └─EncoderBlock: 3-6            30,091,788
|    |    └─EncoderBlock: 3-7            30,091,788
|    |    └─EncoderBlock: 3-8            30,091,788
|    |    └─EncoderBlock: 3-9            30,091,788
|    |    └─EncoderBlock: 3-10           30,091,788
|    |    └─EncoderBlock: 3-11           30,091,788
|    |    └─EncoderBlock: 3-12           30,091,788
|    └─RMSNorm: 2-4                      768
├─Linear: 1-2                            590,592
├─Linear: 1-3                            7,680,000
T

In [32]:

for a in config:
    print(a)

TypeError: 'MoEViTConfig' object is not iterable

In [ ]:
learning_rate = 1e-4
wd = 0.01

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=wd)

In [ ]:
inputs = torch.randn(32, 3, 112, 112).to(current_device)  # Batch size of 32, input size of 784
labels = torch.randint(0, config.num_classes, (32,)).to(current_device)  # Random labels for a batch of 32


In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof:
    with record_function("model_inference"):
        images = inputs
        labels = labels
        logits, embeddings, loss = model(images, labels)
        optimizer.zero_grad()
        # Backpropagation
        loss.backward()
        # Optimizing weights
        optimizer.step()
        # outputs = model(inputs)  # Forward pass
        # loss = criterion(outputs, labels)
        # optimizer.zero_grad()
        # loss.backward()  # Backward pass
        # optimizer.step()

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=40))


In [ ]:
train(model, train_loader, val_loader, id_imgs_loader, 2, optimizer, 6, 'cuda', 'test_cp')

In [ ]:
import ads
from ads.common.auth import default_signer

ads.set_auth(auth='resource_principal')

In [ ]:
import os
import shutil
from glob import glob
import random
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import os
from PIL import Image
import math
import oci

# Initialize a Resource Principals provider
auth_provider = oci.auth.signers.get_resource_principals_signer()

# Create an Object Storage client
object_storage_client = oci.object_storage.ObjectStorageClient({}, signer=auth_provider)

# Get the object
response = object_storage_client.get_object('lr1qux0xfjaq', 'bucket-casia-webface', 'casia-webface.zip')

# Save the object data to a file
with open('data', 'wb') as file:
    for chunk in response.data.raw.stream(1024 * 1024, decode_content=False):
        file.write(chunk)
#print(f"Downloaded {object_name} from bucket {bucket_name} to {download_path}")

In [ ]:
import zipfile
import os

path = f"data"
extract_to = 'data_unzip'

try:
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted all files to {extract_to}")
except FileNotFoundError:
    print(f"Error: The file {path} was not found.")
except zipfile.BadZipFile:
    print(f"Error: The file {path} is not a zip file or it is corrupted.")


In [ ]:
import os

def count_files_in_directory(directory_path):
    """
    Counts the number of files in a directory and its subdirectories.
    
    Parameters:
    directory_path (str): Path to the directory.
    
    Returns:
    int: Number of files in the directory and its subdirectories.
    """
    file_count = 0
    for root, dirs, files in os.walk(directory_path):
        file_count += len(files)
    return file_count

# Example usage
directory_path = 'data_unzip'
number_of_files = count_files_in_directory(directory_path)
print(f"Number of files in '{directory_path}' and its subdirectories: {number_of_files}")